# Baseline: Greedy Placement with Rotation Tightening

This baseline implements:
1. Greedy placement with weighted angle distribution
2. Rotation tightening to minimize bounding box
3. Validation and scoring

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded successfully")

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
    
    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def generate_weighted_angle():
    """Generates a random angle weighted by abs(sin(2*angle)) to favor diagonals."""
    while True:
        angle = random.uniform(0, 2 * math.pi)
        if random.uniform(0, 1) < abs(math.sin(2 * angle)):
            return angle

def get_side_length(trees):
    """Calculate the side length of the bounding square for a list of trees."""
    if not trees:
        return Decimal('0')
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    minx = Decimal(bounds[0]) / scale_factor
    miny = Decimal(bounds[1]) / scale_factor
    maxx = Decimal(bounds[2]) / scale_factor
    maxy = Decimal(bounds[3]) / scale_factor
    width = maxx - minx
    height = maxy - miny
    return max(width, height)

def has_overlap(trees):
    """Check if any two trees overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Helper functions defined")

Helper functions defined


In [4]:
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    if not trees:
        return Decimal('0'), 0.0
    
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    # Use convex hull for efficiency
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun
    
    EPSILON = 1e-8
    if initial_side - found_side > EPSILON:
        return Decimal(str(found_side)), found_angle_deg
    else:
        return Decimal(str(initial_side)), 0.0

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0]) / float(scale_factor)
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print("Rotation optimization functions defined")

Rotation optimization functions defined


In [5]:
def initialize_trees(num_trees, existing_trees=None, num_attempts=10):
    """Greedy placement of trees with weighted angle distribution."""
    if num_trees == 0:
        return [], Decimal('0')

    if existing_trees is None:
        placed_trees = []
    else:
        placed_trees = [t.clone() for t in existing_trees]

    num_to_add = num_trees - len(placed_trees)

    if num_to_add > 0:
        unplaced_trees = [
            ChristmasTree(angle=random.uniform(0, 360)) for _ in range(num_to_add)]
        if not placed_trees:
            placed_trees.append(unplaced_trees.pop(0))

        for tree_to_place in unplaced_trees:
            placed_polygons = [p.polygon for p in placed_trees]
            tree_index = STRtree(placed_polygons)

            best_px = None
            best_py = None
            min_radius = Decimal('Infinity')

            for _ in range(num_attempts):
                angle = generate_weighted_angle()
                vx = Decimal(str(math.cos(angle)))
                vy = Decimal(str(math.sin(angle)))

                radius = Decimal('20.0')
                step_in = Decimal('0.5')

                collision_found = False
                while radius >= 0:
                    px = radius * vx
                    py = radius * vy

                    candidate_poly = affinity.translate(
                        tree_to_place.polygon,
                        xoff=float(px * scale_factor),
                        yoff=float(py * scale_factor))

                    possible_indices = tree_index.query(candidate_poly)
                    if any((candidate_poly.intersects(placed_polygons[i]) and not
                            candidate_poly.touches(placed_polygons[i]))
                           for i in possible_indices):
                        collision_found = True
                        break
                    radius -= step_in

                if collision_found:
                    step_out = Decimal('0.05')
                    while True:
                        radius += step_out
                        px = radius * vx
                        py = radius * vy

                        candidate_poly = affinity.translate(
                            tree_to_place.polygon,
                            xoff=float(px * scale_factor),
                            yoff=float(py * scale_factor))

                        possible_indices = tree_index.query(candidate_poly)
                        if not any((candidate_poly.intersects(placed_polygons[i]) and not
                                   candidate_poly.touches(placed_polygons[i]))
                                   for i in possible_indices):
                            break
                else:
                    radius = Decimal('0')
                    px = Decimal('0')
                    py = Decimal('0')

                if radius < min_radius:
                    min_radius = radius
                    best_px = px
                    best_py = py

            tree_to_place.center_x = best_px
            tree_to_place.center_y = best_py
            tree_to_place.polygon = affinity.translate(
                tree_to_place.polygon,
                xoff=float(tree_to_place.center_x * scale_factor),
                yoff=float(tree_to_place.center_y * scale_factor),
            )
            placed_trees.append(tree_to_place)

    side_length = get_side_length(placed_trees)
    return placed_trees, side_length

print("Greedy placement function defined")

Greedy placement function defined


In [6]:
# Build the index of the submission
index = [f'{n:03d}_{t}' for n in range(1, 201) for t in range(n)]
print(f"Total rows in submission: {len(index)}")

Total rows in submission: 20100


In [7]:
# Run greedy placement with rotation tightening for all N
random.seed(42)  # For reproducibility

all_trees_data = {}  # Store trees for each N
side_lengths = {}  # Store side lengths for each N

start_time = time.time()
current_placed_trees = None

for n in range(1, 201):
    # Greedy placement
    current_placed_trees, side = initialize_trees(n, existing_trees=current_placed_trees, num_attempts=10)
    
    # Rotation tightening
    optimized_side, best_angle = optimize_rotation(current_placed_trees)
    
    if optimized_side < side:
        # Apply the rotation
        current_placed_trees = apply_rotation(current_placed_trees, best_angle)
        side = optimized_side
    
    all_trees_data[n] = current_placed_trees
    side_lengths[n] = side
    
    if n % 20 == 0:
        elapsed = time.time() - start_time
        print(f"N={n:3d}: side={float(side):.6f}, elapsed={elapsed:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s")

N= 20: side=3.990490, elapsed=0.3s


N= 40: side=5.938564, elapsed=0.7s


N= 60: side=7.187462, elapsed=1.2s


N= 80: side=8.073525, elapsed=1.8s


N=100: side=8.928656, elapsed=2.5s


N=120: side=9.595535, elapsed=3.1s


N=140: side=10.640028, elapsed=4.0s


N=160: side=11.384730, elapsed=5.0s


N=180: side=12.127271, elapsed=5.9s


N=200: side=12.847671, elapsed=7.0s

Total time: 7.0s


In [8]:
# Calculate total score
total_score = sum(float(side_lengths[n])**2 / n for n in range(1, 201))
print(f"Total score: {total_score:.6f}")
print(f"Target score: 68.947559")
print(f"Gap: {total_score - 68.947559:.6f}")

Total score: 164.089486
Target score: 68.947559
Gap: 95.141927


In [9]:
# Validate - check for overlaps
print("Validating configurations for overlaps...")
overlap_issues = []
for n in range(1, 201):
    if has_overlap(all_trees_data[n]):
        overlap_issues.append(n)
        
if overlap_issues:
    print(f"WARNING: Overlaps found in N={overlap_issues}")
else:
    print("All configurations valid - no overlaps detected!")

Validating configurations for overlaps...


All configurations valid - no overlaps detected!


In [10]:
# Create submission dataframe
tree_data = []
for n in range(1, 201):
    for tree in all_trees_data[n]:
        tree_data.append([tree.center_x, tree.center_y, tree.angle])

cols = ['x', 'y', 'deg']
submission = pd.DataFrame(
    index=index, columns=cols, data=tree_data).rename_axis('id')

# Round and format
for col in cols:
    submission[col] = submission[col].astype(float).round(decimals=6)
    
# Prepend 's' for string format
for col in submission.columns:
    submission[col] = 's' + submission[col].astype('string')

print(submission.head(20))

                x           y          deg
id                                        
001_0   s0.056435  s-0.305079  s314.999998
002_0   s0.340626  s-0.261989  s346.663915
002_1   s0.002093   s0.119736   s40.667789
003_0   s0.340626  s-0.261989  s346.663915
003_1   s0.002093   s0.119736   s40.667789
003_2   s0.163598   s0.629075  s290.566178
004_0   s0.437512   s0.285729  s433.079933
004_1   s0.035371   s-0.02828  s127.083807
004_2  s-0.462876   s0.164748  s376.982196
004_3   s0.392345  s-0.221006  s223.019977
005_0   s0.404207   s0.320456  s444.374409
005_1   s0.071353  s-0.066231  s138.378283
005_2   s-0.45505   s0.025476  s388.276672
005_3    s0.45916   s-0.18531  s234.314453
005_4   s0.392643   s0.778775  s296.443047
006_0  s-0.134675   s0.316044  s523.098733
006_1   s0.179466  s-0.085994  s217.102607
006_2  s-0.013399  s-0.584305  s467.000997
006_3   s0.372074   s0.271044  s313.038777
006_4  s-0.586408   s0.394317  s375.167372


In [11]:
# Save submission
submission.to_csv('/home/submission/submission.csv')
submission.to_csv('/home/code/experiments/001_baseline/submission.csv')
print("Submission saved!")
print(f"\nFinal score: {total_score:.6f}")

Submission saved!

Final score: 164.089486


In [15]:
# Let's check the sample submission score
import pandas as pd

sample_df = pd.read_csv('/home/data/sample_submission.csv')
print(sample_df.head(20))
print(f"\nTotal rows: {len(sample_df)}")

       id           x           y     deg
0   001_0        s0.0        s0.0   s90.0
1   002_0        s0.0        s0.0   s90.0
2   002_1   s0.202736  s-0.511271   s90.0
3   003_0        s0.0        s0.0   s90.0
4   003_1   s0.202736  s-0.511271   s90.0
5   003_2     s0.5206   s0.177413  s180.0
6   004_0        s0.0        s0.0   s90.0
7   004_1   s0.202736  s-0.511271   s90.0
8   004_2     s0.5206   s0.177413  s180.0
9   004_3  s-0.818657  s-0.228694  s180.0
10  005_0        s0.0        s0.0   s90.0
11  005_1   s0.202736  s-0.511271   s90.0
12  005_2     s0.5206   s0.177413  s180.0
13  005_3  s-0.818657  s-0.228694  s180.0
14  005_4   s0.111852   s0.893022  s180.0
15  006_0        s0.0        s0.0   s90.0
16  006_1   s0.202736  s-0.511271   s90.0
17  006_2     s0.5206   s0.177413  s180.0
18  006_3  s-0.818657  s-0.228694  s180.0
19  006_4   s0.111852   s0.893022  s180.0

Total rows: 20100


In [16]:
# Load sample submission and calculate its score
def load_trees_from_df(df, n):
    """Load trees for a specific N from submission dataframe."""
    group_data = df[df['id'].str.startswith(f'{n:03d}_')]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row['x'])[1:]  # Remove 's' prefix
        y = str(row['y'])[1:]
        deg = str(row['deg'])[1:]
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Calculate sample submission score
sample_score = 0
for n in range(1, 201):
    trees = load_trees_from_df(sample_df, n)
    side = float(get_side_length(trees))
    sample_score += side**2 / n
    
print(f"Sample submission score: {sample_score:.6f}")
print(f"Our baseline score: {total_score:.6f}")
print(f"Target score: 68.947559")

Sample submission score: 173.652299
Our baseline score: 164.089486
Target score: 68.947559


In [14]:
# Calculate sample submission score\nsample_score = 0\nfor n in range(1, 201):\n    trees = load_trees_from_df(sample_df, n)\n    side = float(get_side_length(trees))\n    sample_score += side**2 / n\n    \nprint(f\"Sample submission score: {sample_score:.6f}\")\nprint(f\"Our baseline score: {total_score:.6f}\")\nprint(f\"Target score: 68.947559\")